In [1]:
# === Step 1: Import library ===
import pandas as pd

In [5]:
# === Step 2: Load Dataset dengan Handling Spasi di Header ===
df = pd.read_csv(
    'dataset/Angka-Deforestasi-(Netto)-Indonesia-di-Dalam-dan-di-Luar-Kawasan-Hutan-Tahun-2013-2022-(Ha_Th).csv',
    skipinitialspace=True  # menghilangkan spasi setelah koma
)

# Bersihkan nama kolom dari spasi
df.columns = df.columns.str.strip()

# Cek beberapa baris awal
print(df.head())

  Angka Deforestasi Netto Indonesia Di Dalam Dan Di Luar Kawasan Hutan Tahun 2013-2022  (Ha/Th)  \
0                                                NaN                                              
1                                           Provinsi                                              
2                                                NaN                                              
3                                               Aceh                                              
4                                     Sumatera Utara                                              

      Unnamed: 1                                         Unnamed: 2  \
0            NaN                                                NaN   
1     2013-2014                                                 NaN   
2  Kawasan Hutan  APL (Areal Penggunaan Lain) / Bukan Kawasan Hutan   
3        3,363.9                                            4,284.2   
4        4,675.2                                  

In [ ]:
# === Step 3: Preprocessing (AGGREGATION) ===
# Ubah Data dari Wide ke Long
df_melted = pd.melt(
    df,
    id_vars=['Provinsi'],
    var_name='Tahun_Status',
    value_name='Deforestasi (Ha)'
)

# Pisahkan kolom 'Tahun_Status' menjadi Tahun dan Status Kawasan
df_melted[['Tahun', 'Status Kawasan']] = df_melted['Tahun_Status'].str.extract(r'(\d{4}-\d{4})\s+(.*)')

# Drop kolom gabungan
df_melted = df_melted.drop(columns=['Tahun_Status'])

# Rapikan kolom
df_tidy = df_melted[['Provinsi', 'Tahun', 'Status Kawasan', 'Deforestasi (Ha)']]

print(df_tidy.head())

In [ ]:

# 3.1 Tren Total Deforestasi Nasional per Tahun
tren_total = (
    df.groupby('Tahun', as_index=False)
      .agg({'Deforestasi (Ha)': 'sum'})
)

In [ ]:

# 3.2 Perbandingan Kawasan Hutan vs APL per Tahun
perbandingan_kawasan = (
    df.groupby(['Tahun', 'Status Kawasan'])['Deforestasi (Ha)']
      .sum()
      .unstack(fill_value=0)
      .reset_index()
)

In [ ]:

# 3.3 Akumulasi Deforestasi per Provinsi
deforestasi_provinsi = (
    df.groupby('Provinsi', as_index=False)
      .agg({'Deforestasi (Ha)': 'sum'})
      .sort_values(by='Deforestasi (Ha)', ascending=False)
)

In [ ]:

# 3.4 Deforestasi per Provinsi dan Tahun (Data untuk Heatmap)
heatmap_data = (
    df.pivot_table(
        index='Provinsi',
        columns='Tahun',
        values='Deforestasi (Ha)',
        aggfunc='sum',
        fill_value=0
    )
)

In [ ]:
# === Step 4: Save Aggregated Data to CSV ===

# Membuat folder output jika belum ada
import os
output_folder = 'output'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
tren_total.to_csv(f'{output_folder}/tren_total_deforestasi.csv', index=False)
print("✅ Saved: tren_total_deforestasi.csv")

In [ ]:
perbandingan_kawasan.to_csv(f'{output_folder}/perbandingan_kawasan_hutan_apl.csv', index=False)
print("✅ Saved: perbandingan_kawasan_hutan_apl.csv")

In [ ]:
deforestasi_provinsi.to_csv(f'{output_folder}/deforestasi_akumulasi_provinsi.csv', index=False)
print("✅ Saved: deforestasi_akumulasi_provinsi.csv")

In [ ]:
heatmap_data.to_csv(f'{output_folder}/heatmap_deforestasi_provinsi_tahun.csv')
print("✅ Saved: heatmap_deforestasi_provinsi_tahun.csv")